In [1]:

## CNN_LSTM structure
def CNN_LSTM(q, X_train, Y_train, X_valid, Y_valid, X_test): 
    inputs = tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2]), name='input')
    
    norm = tf.keras.layers.experimental.preprocessing.Normalization()
    norm.adapt(X_train)
    norm_data = norm(inputs)
    x = tf.keras.layers.Conv1D(50, 3, activation='relu', padding='same',
                               kernel_initializer='he_normal')(norm_data)
    x = tf.keras.layers.Conv1D(30, 3, activation='relu', padding='same',
                               kernel_initializer='he_normal')(x)

    x =tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, kernel_initializer='he_normal',
                                                          return_sequences=True,recurrent_dropout=0.3))(x)

    x= tf.keras.layers.Dense(1)(x)
    x1= tf.keras.layers.Flatten()(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=x1)
    tf.keras.utils.plot_model(model, show_shapes=True)
    
    model.compile(loss=lambda y,f: quantile_loss(q,y,f), optimizer=get_opt(1e-2))
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
    history = model.fit(X_train, Y_train, epochs=500, batch_size=16,
                        shuffle=True, validation_data=[X_valid, Y_valid], 
                        callbacks=[es], verbose=0)
    

    train_score = np.asarray(quantile_loss(q,Y_train,model.predict(X_train))).flatten().mean()
    val_score = np.asarray(quantile_loss(q,Y_valid,model.predict(X_valid))).flatten().mean()
    print(train_score, val_score)
    pred = np.asarray(model.predict(X_test))
    return pred, model, val_score

In [2]:
## train and predict Test data
## models for each 4 ML methods and 9 quantiles are trained
def TF_train_func(X_train, Y_train, X_valid, Y_valid, X_test):
    models=[]
    actual_pred = []
    
    for model_select in ['CNN','FFN','LSTM','CNNLSTM'
                 ]:
        score_lst=[]
        for q in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
            print(model_select, q)
            if model_select=='LSTM':
                pred , mod, s = LSTM(q, X_train, Y_train, X_valid, Y_valid, X_test)
            elif model_select=='CNN':
                pred , mod, s = CNN(q, X_train, Y_train, X_valid, Y_valid, X_test)
            elif model_select=='CNNLSTM':
                pred , mod, s = CNN_LSTM(q, X_train, Y_train, X_valid, Y_valid, X_test)
            elif model_select=='FFN':
                pred , mod, s = FFN(q, X_train, Y_train, X_valid, Y_valid, X_test)
            score_lst.append(s)
            models.append(mod)
            actual_pred.append(pred)
        print(sum(score_lst)/len(score_lst))
    return models, np.asarray(actual_pred)

In [3]:
print(TF_X_train.shape) 
print(TF_Y_train_1.shape) 
print(TF_X_valid.shape) 
print(TF_Y_valid_1.shape)
print(tf_test.shape)

## train model. prediction for day7 and day 8 are sperately handled by distinct model.
## total of 4*9*2 models and trained. each for 4 model types, 9 quantiles and 2 seperate target days
models_tf1, results_tf1 = TF_train_func(TF_X_train, TF_Y_train_1 , TF_X_valid, TF_Y_valid_1, tf_test)
models_tf2, results_tf2 = TF_train_func(TF_X_train, TF_Y_train_2 , TF_X_valid, TF_Y_valid_2, tf_test)

NameError: name 'TF_X_train' is not defined